# 黑色金属 - 螺纹钢

In [5]:
# 设置工作目录
import os
os.getcwd()
if os.name == 'posix':
    os.chdir('/Volumes/Repository/Projects/ffa/')
else:
    os.chdir("E:\\Document\\Project\\ffa")

In [6]:
# 加载依赖模块
import pandas as pd
import numpy as np
import akshare as ak
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date
import importlib
import commodity
import json

In [17]:
# 重新加载salary计算模块
importlib.reload(commodity)

<module 'commodity' from '/Volumes/Repository/Projects/ffa/commodity.py'>

## 数据准备与预处理

In [8]:
symbol_id = 'RB'
symbol_name = '螺纹钢'
fBasePath = 'steel/data/mid-stream/螺纹钢/'
json_file = './steel/setting.json'

### 数据索引设置

In [13]:
# 品种数据索引初始化
# 首次使用json配置文件存取品种的数据索引
data_index = {'主力合约': {'Name': "dominant_contract", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'dominant_contract', 'DataFrame': "df_basis"},
                '近月合约': {'Name': "near_contract", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'near_contract', 'DataFrame': "df_basis"},
                '主力合约收盘价': {'Name': "dominant_close_price", 'Source':'Choice', 'Path': fBasePath + '螺纹钢期货价格.xlsx', 'Field':'期货收盘价（主力）：螺纹钢', 'DataFrame': "df_dominant"},
                '主力合约结算价': {'Name': "dominant_settle_price", 'Source':'Choice', 'Path': fBasePath + '螺纹钢期货价格.xlsx', 'Field':'期货结算价（主力）：螺纹钢', 'DataFrame': "df_dominant"},
                '现货价格': {'Name': "spot_price", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'spot_price', 'DataFrame': "df_basis"},
                '基差': {'Name': "basis", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'dom_basis', 'DataFrame': "df_basis"},
                '基差率': {'Name': "basis_rate", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'dom_basis_rate', 'DataFrame': "df_basis"},
                '产量': {'Name': "production", 'Source':'Choice', 'Path': fBasePath + '螺纹钢产量.xlsx', 'Field':'产量：钢筋：全国：当月值', 'DataFrame': "df_prodoction"},
                '销量': {'Name': "sales", 'Source':'Choice', 'Path': fBasePath + '螺纹钢销量.xlsx', 'Field':'销量：钢筋：累计值', 'DataFrame': "df_sales"},
                '库存': {'Name': "instock", 'Source':'Choice', 'Path': fBasePath + '螺纹钢库存.xlsx', 'Field':'库存：螺纹钢：合计', 'DataFrame': "df_instock"},
                # TODO: 校验Choice提供的另外一套库存和仓单数据
                '仓单': {'Name': "receipt", 'Source':'Choice', 'Path': fBasePath + '螺纹钢库存.xlsx', 'Field':'仓单数量：螺纹钢', 'DataFrame': "df_instock"}
                }
symbol_setting = {'DataIndex': data_index,
                  '利润公式': {'Name': 'profit_formula', '铁矿': 1.6, '焦炭': 0.6, '其他成本': 1200}}


In [18]:
# 构造品种数据访问对象
# symbol = commodity.SymbolData(symbol_id, symbol_name, json_file, symbol_setting)
symbol = commodity.SymbolData(symbol_id, symbol_name, json_file)
# merged_data = symbol.merge_data()

In [11]:
symbol_j = commodity.SymbolData('J', '焦炭', json_file)
data_j = symbol_j.merge_data()
symbol_i = commodity.SymbolData('I', '铁矿石', json_file)
data_i = symbol_i.merge_data()

In [ ]:
ranked_list = ['库存', '仓单']
df_rank = pd.DataFrame()
for field in ranked_list:
    df_rank = symbol.history_time_ratio(field, df_rank)
df_rank

### 更新数据

- 通过AKshare接口更新现货价格数据
- 手动更新Choice导出数据（使用Microsoft Excel打开，自动更新后保存）

In [7]:
# 更新现货价格数据 - AKShare
# 追加最新数据
df_basis = symbol.update_akshare_file(mode='append')
# 2011、2012年数据待更新
# symbol_data.update_akshare_file(mode='period', start_date='20120101', end_date='20121231')

/Volumes/Repository/app/miniconda3/lib/python3.10/site-packages/akshare/futures/futures_basis.py:101: UserWarning: 20231202非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")


## 基差-库存/仓单-利润分析

### 历史走势分析

前置条件：
- 非季节性品种确认

分析内容：
- 现货价格/期货价格（收盘价）、基差的历史趋势
- 基差率历史趋势，基差率历史分位
- 库存、仓单、库存消费比，库存、仓单的历史分位
- 现货利润和盘面利润，及其历史分位
- 现货月区域标记
- 多维指标共振区域标记

扩展功能：
- 图表可配置化
- 小图：时点跨期套利分析
- 小图：时点期限结构分析

In [ ]:
# 图表配置信息
figure_dict = {'基础属性':{'title':'基差-库存-利润分析', 'width':'auto', 'height':800, 'slider':False, 'selector':True},
               '主图':{'title':'基差', 'height':0.7, 'x_axis':symbol.symbol_data['日期'], 
                      'y_axis':{'期货收盘价': {'type':'Scatter', 'color':'rgb(84,134,240)', 'secondary_y':False},
                                '现货价格':{'type':'Scatter', 'color':'rgb(84,134,240)', 'secondary_y':False},
                                '基差':{'type':'Area', 'color':'rgb(239,181,59)', 'secondary_y':True, 'stackgroup':'one'}},
               '辅图':{'库存':{'title':'基差', 'height':0.7, 'x_axis':symbol.symbol_data['日期'],
                      'y_axis':{'库存':{'type':'Bar', 'color':'rgb(84,134,240)', 'secondary_y':False}}},
                      '利润':{}}
               }}

In [11]:
# 通过AKShare获取的基差/基差率数据（螺纹钢）出现正负符号错误，注意多次运行会不断反转
symbol.symbol_data['基差'] = 0-symbol.symbol_data['基差']
symbol.symbol_data['基差率'] = 0-symbol.symbol_data['基差率']
df_figure = pd.DataFrame()
df_figure['基差率颜色'] = symbol.symbol_data['基差率'] > 0
df_figure['基差率颜色'] = df_figure['基差率颜色'].replace({True:1, False:0})

max_y = symbol.symbol_data['主力合约收盘价'] .max() * 1.05
min_y = symbol.symbol_data['主力合约收盘价'] .min() * 0.95

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
                   vertical_spacing=0.01, subplot_titles=('基差分析', '基差率', '库存', '库存历史分位'), 
                   row_width=[0.1, 0.1, 0.1, 0.7])

#fig_main = make_subplots(specs=[[{"secondary_y": True}]])
fig_future_price = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['主力合约收盘价'], name='期货价格', marker_color='rgb(84,134,240)')
fig_spot_price = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['现货价格'], name='现货价格', marker_color='rgb(105,206,159)')
fig_basis = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['基差'], stackgroup='one', name='基差', marker_color='rgb(239,181,59)')
#fig_main.add_trace(fig_future_price)
#fig_main.add_trace(fig_basis_rate, secondary_y=True)

fig.add_trace(fig_basis, secondary_y=True)
fig.add_trace(fig_future_price, row = 1, col = 1)
fig.add_trace(fig_spot_price, row = 1, col = 1)

fig_basis_rate = go.Bar(x=symbol.symbol_data['日期'], y = symbol.symbol_data['基差率'], name='基差率', marker_color='rgb(244,128,68)')
#fig_basis_rate = go.Bar()
#fig_basis_rate.mode = 'markers'
# fig_basis_rate.x = df_rb0['日期']
# fig_basis_rate.y = df_rb0['基差率']
fig_basis_rate.marker.colorscale = ['green', 'red']
fig_basis_rate.marker.color = df_figure['基差率颜色']
#fig_basis_rate.marker.size = 2
fig_basis_rate.marker.showscale = False
fig_basis_rate.showlegend = False

fig.add_trace(fig_basis_rate, row = 2, col = 1)
fig_receipt = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['仓单'], name='仓单', marker_color='rgb(239,181,59)')
fig_storage = go.Bar(x=symbol.symbol_data['日期'], y=symbol.symbol_data['库存'], name='库存', marker_color='rgb(234,69,70)')
#fig_storage = px.bar(df_rb0, x='日期', y='库存')
fig.add_trace(fig_receipt, row = 3, col = 1, secondary_y=True)
fig.add_trace(fig_storage, row = 3, col = 1)

fig_receipt_rank = go.Scatter(x=symbol.symbol_data['日期'], y=df_rank['仓单历史时间百分位'], name='仓单分位', marker_color='rgb(239,181,59)')
fig_storage_rank = go.Bar(x=symbol.symbol_data['日期'], y=df_rank['库存历史时间百分位'], name='库存分位', marker_color='rgb(234,69,70)')
#fig_storage = px.bar(df_rb0, x='日期', y='库存')
fig.add_trace(fig_receipt_rank, row = 4, col = 1, secondary_y=True)
fig.add_trace(fig_storage_rank, row = 4, col = 1)

trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
dt_all = pd.date_range(start=symbol.symbol_data['日期'].iloc[0],end=symbol.symbol_data['日期'].iloc[-1])
dt_all = [d.strftime("%Y-%m-%d") for d in dt_all]
dt_breaks = list(set(dt_all) - set(trade_date))

# X轴坐标按照年-月显示
fig.update_xaxes(
    showgrid=True,
    zeroline=True,
    dtick="M1",  # 按月显示
    ticklabelmode="period",   # instant  period
    tickformat="%b\n%Y",
    rangebreaks=[dict(values=dt_breaks)],
    rangeslider_visible = False, # 下方滑动条缩放
    rangeselector = dict(
        # 增加固定范围选择
        buttons = list([
            dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
            dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
            dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
            dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
            dict(step = 'all')
            ]))
)
#fig.update_traces(xbins_size="M1")
fig.update_layout(
    yaxis_range=[min_y,max_y],
    #autosize=False,
    #width=800,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.show()

### 历史水位分析

基差率-库存消费比-利润率

历史时间比例分位

In [ ]:
# df1 = symbol.history_time_ratio('库存', df_rank=merged_data)
# df2 = symbol.history_time_ratio('库存', df_rank=merged_data, mode='value')


In [ ]:
# 将累计销量数据转化为当月销量数据

# 假设您提供的数据保存在一个名为df的dataframe中，字段包含日期和累计销量
# 例如，df的前五行如下：
#          日期   累计销量
# 0 2020-01-31  1000
# 1 2020-02-29  1500
# 2 2020-03-31  1800
# 3 2020-04-30  2200
# 4 2020-05-31  2500

# 定义一个函数，计算当月销量值
def calc_monthly_sales(df):
    # 创建一个空的列表，用于存储当月销量值
    monthly_sales = []
    # 遍历dataframe的每一行，获取日期和累计销量
    for i, row in df.iterrows():
        # 获取日期
        date = row['日期']
        # 获取累计销量
        cum_sales = row['累计销量']
        # 如果是第一行，那么需要判断是否是1月份
        if i == 0:
            # 如果是1月份，那么当月销量值就等于累计销量
            if date.month == 1:
                monthly_sales.append(cum_sales)
            # 如果不是1月份，那么当月销量值就设为NaN
            else:
                monthly_sales.append(np.nan)
        # 如果不是第一行，那么需要判断当前月份与上一行的月份是否相邻
        else:
            # 获取上一行的日期
            prev_date = df.loc[i-1, '日期']
            # 如果当前月份与上一行的月份相邻，那么当月销量值就等于累计销量减去上一行的累计销量
            if date.month == prev_date.month + 1 or (date.month == 1 and prev_date.month == 12):
                monthly_sales.append(cum_sales - df.loc[i-1, '累计销量'])
            # 如果当前月份与上一行的月份不相邻，那么当月销量值就设为NaN
            else:
                monthly_sales.append(np.nan)
    # 返回列表
    return monthly_sales

# 调用函数，得到一个列表，存储当月销量值
monthly_sales = calc_monthly_sales(df)

# 在原始的dataframe中，创建一个新的列，存储当月销量值
df['当月销量'] = monthly_sales

# 打印dataframe的前五行，查看结果
print(df.head())

#          日期   累计销量  当月销量
# 0 2020-01-31  1000   1000.0
# 1 2020-02-29  1500    500.0
# 2 2020-03-31  1800    300.0
# 3 2020-04-30  2200    400.0
# 4 2020-05-31  2500    300.0


In [ ]:
# 使用pandas中的fillna方法，使用上一个非NaN的“销量”数据填充后面日期的NaN
# fillna方法会根据给定的参数，将数据中的NaN值替换为其他的值
# method参数可以指定填充的方式，我们可以设置为'ffill'，表示使用上一个非NaN的值进行填充
# limit参数可以指定填充的次数，我们可以设置为None，表示不限制填充的次数
df['销量'] = df['销量'].fillna(method='ffill', limit=None)

## 季节性分析

### 基差率季节分析

In [ ]:
df_rb0['年度'] = df_rb0['日期'].dt.year
df_rb0['年内日期'] = df_rb0['日期'].dt.strftime('1900-%m-%d')
fig_basis_rate_season = px.line(df_rb0,
                                x='年内日期',
                                y='基差率',
                                color='年度',
                                #color_discrete_sequence=px.colors.qualitative.G10)
                                color_discrete_sequence=['lightgray', 'lightblue', 'orange', 'red'])
fig_basis_rate_season.update_layout(
    title={
        'text':'基差率季节分析',
        'xanchor':'center'},
    margin=dict(l=10, r=10, t=40, b=10)
)

fig_basis_rate_season.show()

### 基差率月度涨跌统计

### 基差率频率分布

### 库存季节性分析

## 跨期分析

### 期限结构

In [ ]:
# 加载合约基础数据
futures_comm_info = pd.read_excel('data/common_info.xlsx')
spec_contact_list = futures_comm_info[futures_comm_info.合约名称.str.startswith('螺纹钢')]
fig_term = make_subplots(specs=[[{"secondary_y": True}]])
fig_term.add_trace(go.Scatter(x=spec_contact_list['合约代码'], y=spec_contact_list['现价']))
# 获取最新现货价格
spot_price = df_rb0[df_rb0['现货']!=0]['现货'].iloc[-1]
fig_term.add_hline(y=spot_price)
fig_term.update_layout(
    title={
        'text':'期限结构'
    },
    #autosize=False,
    width=800,
    #height=800,
    margin=dict(l=10, r=10, t=40, b=10)
)
fig_term.show()

### 套利分析

#### 价差分析-多期排列

#### 价差分析-跨期价差矩阵

#### 基差-月差分析

#### 价差季节性分析

## 库存

### 库存周期

#### 期转现

#### 交割统计

## 利润

### 现货利润

### 期货盘面利润

### 利润期限结构

## 综合分析

### 基差-库存-利润分析

### 基差-月差分析

### 期限结构-库存/仓单分析